In [1]:
#loading libraries 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn. preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df_train=pd.read_csv("Train_Clean.csv")
df_test=pd.read_csv("Test_Clean.csv")


In [10]:
x_train=df_train["Tweets"]
y_train=df_train["S/G"]
x_test=df_test["Tweet"]
y_test=df_test["G/S"]
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((3564,), (935,), (3564,), (935,))

In [11]:
#vectorizing the urdu statements using the Term frequency and inverse document frequency
from sklearn.feature_extraction.text import TfidfVectorizer
max_feature_num = 50000
vectorizer = TfidfVectorizer(max_features=max_feature_num)
train_vecs_2 = vectorizer.fit_transform(x_train)
test_vecs_2 = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).fit_transform(x_test)
# visualizing the shape of train and test vecs
train_vecs.shape,test_vecs.shape


((3564, 7642), (935, 7642))

In [ ]:
from lazypredict.Supervised import LazyClassifier
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(train_vecs.toarray(),test_vecs.toarray(),np.array(y_train),np.array(y_test))
models

In [13]:
estimators=[]
#adding weak 6 gnb
estimators.append(("GNB1",GaussianNB()))
estimators.append(("GNB2",GaussianNB()))
estimators.append(("GNB3",GaussianNB()))
estimators.append(("GNB4",GaussianNB()))
estimators.append(("GNB5",GaussianNB()))
estimators.append(("GNB6",GaussianNB()))
#adding 6 weak svc 
estimators.append(("LSVC1",LinearSVC()))
estimators.append(("LSVC2",LinearSVC()))
estimators.append(("LSVC3",LinearSVC()))
estimators.append(("LSVC4",LinearSVC()))
estimators.append(("LSVC5",LinearSVC()))
estimators.append(("LSVC6",LinearSVC()))
#adding 6 weak dtc
estimators.append(("DTC1",DecisionTreeClassifier()))
estimators.append(("DTC2",DecisionTreeClassifier()))
estimators.append(("DTC3",DecisionTreeClassifier()))
estimators.append(("DTC4",DecisionTreeClassifier()))
estimators.append(("DTC5",DecisionTreeClassifier()))
estimators.append(("DTC6",DecisionTreeClassifier()))
#adding 6 weak knn
estimators.append(("KNN1",KNeighborsClassifier()))
estimators.append(("KNN2",KNeighborsClassifier()))
estimators.append(("KNN3",KNeighborsClassifier()))
estimators.append(("KNN4",KNeighborsClassifier()))
estimators.append(("KNN5",KNeighborsClassifier()))
estimators.append(("KNN6",KNeighborsClassifier()))
#adding 6 weak logistic regression classifier
estimators.append(("LR1",LogisticRegression()))
estimators.append(("LR2",LogisticRegression()))
estimators.append(("LR3",LogisticRegression()))
estimators.append(("LR4",LogisticRegression()))
estimators.append(("LR5",LogisticRegression()))
estimators.append(("LR6",LogisticRegression()))
#adding weak Multinomial Naive Bayes Classifier
estimators.append(("MNB1",MultinomialNB()))
estimators.append(("MNB2",MultinomialNB()))
estimators.append(("MNB3",MultinomialNB()))
estimators.append(("MNB4",MultinomialNB()))
estimators.append(("MNB5",MultinomialNB()))
estimators.append(("MNB6",MultinomialNB()))
#adding 6 weak LDA classfiers 
estimators.append(("LDA1",LinearDiscriminantAnalysis()))
estimators.append(("LDA2",LinearDiscriminantAnalysis()))
estimators.append(("LDA3",LinearDiscriminantAnalysis()))
estimators.append(("LDA4",LinearDiscriminantAnalysis()))
estimators.append(("LDA5",LinearDiscriminantAnalysis()))
estimators.append(("LDA6",LinearDiscriminantAnalysis()))

In [17]:
#defining the ensemble model
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
np.random.seed(42)
def build_model(model,xtrain,ytrain,xtest,ytest):
    clf = model
    clf.fit(xtrain,ytrain)
    clf_predictions = clf.predict(xtest)
    acc = accuracy_score(ytest,clf_predictions)
    f_1_score = f1_score(ytest,clf_predictions,average="weighted")
#     roc_auc=roc_auc_score(ytest,clf_predictions,multi_class="ovo")
    result = {"accuracy:":acc,"f1_score":f_1_score}
    return model,result
ensemble_model,result=build_model(VotingClassifier(estimators),train_vecs.toarray(),y_train,test_vecs.toarray(),y_test)

In [18]:
ensemble_model

VotingClassifier(estimators=[('GNB1', GaussianNB()), ('GNB2', GaussianNB()),
                             ('GNB3', GaussianNB()), ('GNB4', GaussianNB()),
                             ('GNB5', GaussianNB()), ('GNB6', GaussianNB()),
                             ('LSVC1', LinearSVC()), ('LSVC2', LinearSVC()),
                             ('LSVC3', LinearSVC()), ('LSVC4', LinearSVC()),
                             ('LSVC5', LinearSVC()), ('LSVC6', LinearSVC()),
                             ('DTC1', DecisionTreeClassifier()),
                             ('DTC2', DecisionTreeCla...
                             ('KNN1', KNeighborsClassifier()),
                             ('KNN2', KNeighborsClassifier()),
                             ('KNN3', KNeighborsClassifier()),
                             ('KNN4', KNeighborsClassifier()),
                             ('KNN5', KNeighborsClassifier()),
                             ('KNN6', KNeighborsClassifier()),
                             ('LR1', LogisticRegression()),
                             ('LR2', LogisticRegression()),
                             ('LR3', LogisticRegression()),
                             ('LR4', LogisticRegression()),
                             ('LR5', LogisticRegression()),
                             ('LR6', LogisticRegression()), ...])

In [19]:
result

{'accuracy:': 0.5422459893048128, 'f1_score': 0.5265703240572425}

In [24]:
#cross validation of ensemble technique
from sklearn.model_selection import KFold, cross_val_score
kf =KFold(n_splits=10, shuffle=True, random_state=42)
result_cv_acc=cross_val_score(ensemble_model,train_vecs.toarray(),y_train,scoring="accuracy",cv=kf)

In [25]:
result_cv_acc

array([0.61344538, 0.66386555, 0.61904762, 0.61904762, 0.64325843,
       0.67696629, 0.61235955, 0.66011236, 0.61516854, 0.68258427])

In [26]:
result_cv_acc.mean()

0.6405855600667233

In [27]:
result_cv_f1=cross_val_score(ensemble_model,train_vecs.toarray(),y_train,scoring="f1",cv=kf)

In [28]:
result_cv_f1

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])